In [63]:
import os, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [64]:
all_files = os.listdir('results')
cfgs = sorted([n for n in all_files if 'cfg' in n])
perfs = sorted([n[:-11]+'_performance_summary.csv' for n in cfgs])

In [65]:
list(zip(cfgs, perfs))

[]

In [15]:
pd.read_csv('results/'+perfs[0])

,dataset,auc,acc,dice/F1,spec,sens
0,DRIVE_train,0.9545,0.9397,0.7567,0.9681,0.7429
1,DRIVE_val,0.9641,0.9442,0.7691,0.9695,0.7620
2,DRIVE,0.9577,0.9453,0.7631,0.9821,0.6925
3,STARE,0.9383,0.9434,0.6794,0.9850,0.5816
4,CHASEDB,0.9289,0.9341,0.6557,0.9693,0.6209
5,AV-WIDE,0.7449,0.9394,0.2400,0.9962,0.1436


In [16]:
all_results = []
for i in range(len(cfgs)):
    with open('results/'+cfgs[i], 'r') as f:
        args = json.load(f)
    res = pd.read_csv('results/'+perfs[i])    
    all_results.append([args,res])

In [17]:
list(all_results[0][0].keys())

['layers',
 'n_classes',
 'in_channels',
 'up_mode',
 'pool_mode',
 'conv_bridge',
 'shortcut',
 'lr',
 'disable_transforms',
 'eps',
 'batch_size',
 'data_aug',
 'normalize',
 'metric',
 'patience',
 'scheduler_f',
 'n_epochs',
 'data_path',
 'end2end',
 'experiment_path',
 'config_file_path']

[8/16/32,4/16/32,4/8/64,4/8/32,4/8/16/32]

In [62]:
both_of_them_l = []
conv_bridge_l = []
shortcut_l = []
none_of_them_l = []
for i in range(len(cfgs)):
#     if all_results[i][0]['layers']=='4/8/16/32':
#     if all_results[i][0]['layers']=='4/8/32':
#     if all_results[i][0]['layers']=='4/8/64':
#     if all_results[i][0]['layers']=='4/16/32':
    if all_results[i][0]['layers']=='8/16/32':
        conv_bridge = all_results[i][0]['conv_bridge']
        shortcut = all_results[i][0]['shortcut']
        lr = all_results[i][0]['lr']
        val_auc = all_results[i][1].iat[1,1]
        train_auc = all_results[i][1].iat[0,1]
        test_auc = all_results[i][1].iat[2,1]
        if val_auc == 0.9828: 
            print('this one for none of them!')
            print(all_results[i][0])
            print(all_results[i][1])
        if val_auc == 0.9823: 
            print('this one for both of them!', lr)
            print(all_results[i][0])
            print(all_results[i][1])
        if conv_bridge and shortcut:
            both_of_them_l.append([lr,val_auc,test_auc])
        if conv_bridge and not shortcut:
            conv_bridge_l.append([lr,val_auc,test_auc])
        if not conv_bridge and shortcut:
            shortcut_l.append([lr,val_auc,test_auc])       
        if not conv_bridge and not shortcut:
            none_of_them_l.append([lr,val_auc,test_auc])

both_of_them = np.array(both_of_them_l)
conv_bridge = np.array(conv_bridge_l)
shortcut = np.array(shortcut_l)
none_of_them = np.array(none_of_them_l)

this one for none of them!
{'layers': '8/16/32', 'n_classes': 1, 'in_channels': 3, 'up_mode': 'transp_conv', 'pool_mode': 'max_pool', 'conv_bridge': False, 'shortcut': False, 'lr': 0.01, 'disable_transforms': False, 'eps': 1e-08, 'batch_size': 4, 'data_aug': 'all', 'normalize': 'from_im_max', 'metric': 'auc', 'patience': 100, 'scheduler_f': 0.1, 'n_epochs': 1000, 'data_path': 'data/DRIVE/', 'end2end': False, 'experiment_path': 'experiments/2020-01-09-16:2054', 'config_file_path': 'experiments/2020-01-09-16:2054/config.cfg'}
       dataset     auc     acc  dice/F1    spec    sens
0  DRIVE_train  0.9807  0.9563   0.8271  0.9747  0.8284
1    DRIVE_val  0.9828  0.9548   0.8182  0.9714  0.8351
2        DRIVE  0.9795  0.9561   0.8203  0.9807  0.7875
3        STARE  0.9720  0.9571   0.7883  0.9780  0.7749
4      CHASEDB  0.9719  0.9489   0.7582  0.9664  0.7928
5      AV-WIDE  0.8777  0.9551   0.5999  0.9872  0.5053
this one for both of them! 0.01
{'layers': '8/16/32', 'n_classes': 1, 'in_chan

In [56]:
none_of_them

array([[0.01  , 0.9641, 0.9577],
       [0.01  , 0.637 , 0.587 ],
       [0.01  , 0.9639, 0.9588],
       [0.01  , 0.9828, 0.9795],
       [0.05  , 0.9814, 0.9782],
       [0.005 , 0.9826, 0.9793]])

In [53]:
none_of_them[none_of_them[:,0].argsort()]

array([[0.005 , 0.9826, 0.9793],
       [0.01  , 0.9641, 0.9577],
       [0.01  , 0.637 , 0.587 ],
       [0.01  , 0.9639, 0.9588],
       [0.01  , 0.9828, 0.9795],
       [0.05  , 0.9814, 0.9782]])

In [54]:
both_of_them[both_of_them[:,0].argsort()]

array([[0.005 , 0.9823, 0.979 ],
       [0.01  , 0.9496, 0.9462],
       [0.01  , 0.9823, 0.9787],
       [0.05  , 0.9821, 0.9787]])

In [ ]:
# conv_bridge[conv_bridge[:,0].argsort()]

In [ ]:
# shortcut[shortcut[:,0].argsort()]

In [55]:
all_results[0][1]

,dataset,auc,acc,dice/F1,spec,sens
0,DRIVE_train,0.9545,0.9397,0.7567,0.9681,0.7429
1,DRIVE_val,0.9641,0.9442,0.7691,0.9695,0.7620
2,DRIVE,0.9577,0.9453,0.7631,0.9821,0.6925
3,STARE,0.9383,0.9434,0.6794,0.9850,0.5816
4,CHASEDB,0.9289,0.9341,0.6557,0.9693,0.6209
5,AV-WIDE,0.7449,0.9394,0.2400,0.9962,0.1436
